<a href="https://colab.research.google.com/github/MuhammadAbuSufian/Evaluation-of-ImageReward-and-PickScore-Models-on-AIGIQA-4K-Dataset/blob/main/AIGIQA_PickScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
from transformers import AutoProcessor, AutoModel
from PIL import Image
import torch
from torch.nn import functional as F
from io import BytesIO
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau
import os
import pandas as pd
from google.colab import drive

In [ ]:
# Setup the device for model computation
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model and processor from Hugging Face
processor_name_or_path = "laion/CLIP-ViT-H-14-laion2B-s32B-b79K"
model_pretrained_name_or_path = "yuvalkirstain/PickScore_v1"
processor = AutoProcessor.from_pretrained(processor_name_or_path)
model = AutoModel.from_pretrained(model_pretrained_name_or_path).eval().to(device)

In [ ]:
def calc_alignment_score(prompt, image):

    image_input = processor(images=image, return_tensors="pt").to(device)
    text_input = processor(text=prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        image_emb = model.get_image_features(**image_input)
        image_emb = image_emb / image_emb.norm(dim=-1, keepdim=True)

        text_emb = model.get_text_features(**text_input)
        text_emb = text_emb / text_emb.norm(dim=-1, keepdim=True)

        match_score = (text_emb @ image_emb.T).squeeze()

    return match_score.item()

In [ ]:
drive.mount('/content/drive')

# Load the CSV file
raw_data = pd.read_csv('/content/drive/MyDrive/Colab/AIGIQA/annotation.csv')
data = raw_data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pick_scores = []
AIGIQA_socres = []

for index, row in data.iterrows():

    image_name = row['Generated_image']
    prompt = row['Text_prompt']
    AIGIQA_socre = row['MOS_a']

    image_path = os.path.join('/content/drive/MyDrive/Colab/AIGIQA/All', image_name)

    try:
        with Image.open(image_path) as img:
            reward = calc_alignment_score(prompt, img)
            pick_scores.append(reward)
            AIGIQA_socres.append(AIGIQA_socre.split('_')[0])
    except IOError:
        print(f"Failed to load image {image_name}")

In [ ]:
subjective_scores = np.array(AIGIQA_socres, dtype=float)
objective_scores = np.array(pick_scores, dtype=float)

plcc, _ = pearsonr(subjective_scores, objective_scores)
srocc, _ = spearmanr(subjective_scores, objective_scores)
krocc, _ = kendalltau(subjective_scores, objective_scores)

print("PLCC:", plcc)
print("SROCC:", srocc)
print("KROCC:", krocc)


Sizes: 4000 4000
PLCC: 0.35643929447598377
SROCC: 0.35939199067030436
KROCC: 0.24244184537158392
